# Site Selection for Japanese Restaurant in Toronto

## Table of Contents

1.Download and Explore Dataset

2.Explore Neighborhoods in Toronto

3.Analyze Each Neighborhood

4.Cluster Neighborhoods 

5.Examine Clusters to Select the Best Location

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
from urllib.request import urlopen

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

Scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the list of postal codes of Toronto. 

In [2]:
# parse the wikipedia page and transform the data into a dataframe df

html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

soup = BeautifulSoup(html, features='lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

data = []

for record in My_table.findAll('td'):
    data.append(record.text)

PostalCode = data[0::3]
Borough = data[1::3]
Neighborhood = data[2::3]
Neighborhood = [line.rstrip('\n') for line in Neighborhood]

df = pd.DataFrame()
df['PostalCode'] = PostalCode
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood

df = df[df.Borough != 'Not assigned']

df = df.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(','.join)

# for Borough "Queen's Park", replace its Neighborhood column value with "Queen's Park"

df.replace('Not assigned', 'Queen\'s Park', inplace = True)

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [3]:
# read the csv file that contains the geographical coordinates of each postal code in Toronto

geo = pd.read_csv('geo.csv')

# rename the column "Postal Code" to "PostalCode"
geo.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

# merge the new dataframe toronto with the previous dataframe df

MetroToronto = pd.merge(df, geo, on = "PostalCode")

MetroToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(MetroToronto['Borough'].unique()),
        MetroToronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Since we are only interested in boroughs that contain the word Toronto, we will slice the original dataframe and create a new dataframe of the Toronto data. 

In [5]:
trt = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
toronto = MetroToronto[MetroToronto.Borough.isin(trt)].reset_index(drop=True)

toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Use geopy library to get the latitude and longitude values of Toronto.

In [6]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Visualize the neighborhoods in Toronto.

In [7]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

Utilize the Foursquare API to explore the neighborhoods and segment them.

In [8]:
CLIENT_ID = 'JGZNWOCWTNWGTNXRJPB3CZWBRYXARMEXVI4PY2WVMY4NEVOX' # your Foursquare ID
CLIENT_SECRET = 'J0IFJCEHRZYBRHKUWC2ZIC45R0AP3I0LRNCSW1VNRXAWUDRV' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JGZNWOCWTNWGTNXRJPB3CZWBRYXARMEXVI4PY2WVMY4NEVOX
CLIENT_SECRET:J0IFJCEHRZYBRHKUWC2ZIC45R0AP3I0LRNCSW1VNRXAWUDRV


Create a function to get nearby venues of all the neighborhoods in Toronto. We will get 100 venues within the radius of 2500 meters for each neighborhood.

In [9]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called "toronto_venues".

In [10]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [11]:
print('A total of {} venues were returned for all of the neighborhoods in Toronto.'.format(
        toronto_venues.shape[0]
    )
)

A total of 3800 venues were returned for all of the neighborhoods in Toronto.


In [12]:
toronto_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Historic Site', 'Farmers Market',
       'Mediterranean Restaurant', 'Chocolate Shop', 'Restaurant',
       'Gym / Fitness Center', 'Dessert Shop', 'Park',
       'Italian Restaurant', 'Performing Arts Venue', 'Liquor Store',
       'Thai Restaurant', 'Café', 'French Restaurant', 'Pool', 'Hotel',
       'Diner', 'Middle Eastern Restaurant', 'BBQ Joint',
       'Athletics & Sports', 'German Restaurant', 'Bistro',
       'Breakfast Spot', 'Grocery Store', 'Gastropub', 'Food Truck',
       'Seafood Restaurant', 'Japanese Restaurant', 'Curling Ice',
       'Creperie', 'Taco Place', 'Bar', 'Circus', 'Cosmetics Shop',
       'Theater', 'Pub', 'Steakhouse', 'Farm', 'Garden',
       'American Restaurant', 'Church', 'Beer Bar', 'Burrito Place',
       'Cocktail Bar', 'Museum', 'Ramen Restaurant', 'Gym',
       'Clothing Store', 'Pizza Place', 'Ice Cream Shop',
       'Indian Restaurant', 'Jewelry Store', 'Shopping Mall',
       'Vietnamese Restaurant', 'Sandwich

In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


## 3. Analyze Each Neighborhood

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.00,0.01,0.0,0.0,0.01,0.02,0.0,0.01,0.00,0.00,0.01,0.0,0.0,0.00,0.0,0.00,0.0,0.02,0.01,0.00,0.03,0.0,0.00,0.01,0.00,0.01,0.01,0.01,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.04,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.09,0.01,0.00,0.01,0.02,0.0,0.01,0.00,0.01,0.00,0.0,0.00,0.01,0.0,0.0,0.01,0.02,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.03,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.05,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.03,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.01,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.03,0.0,0.02,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.01,0.02,0.0,0.01,0.00,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.01,0.01,0.02,0.0,0.0,0.02,0.0,0.00,0.0

We will consider both "Japanese Restaurant" and "Sushi Restaurant" as Japanese Restaurant. Create a new dataframe for Japanese Restaurant data only. 

In [16]:
jpn = toronto_grouped[["Neighborhood","Japanese Restaurant","Sushi Restaurant"]]
jpn.head()

,Neighborhood,Japanese Restaurant,Sushi Restaurant
0,"Adelaide,King,Richmond",0.03,0.01
1,Berczy Park,0.02,0.01
2,"Brockton,Exhibition Place,Parkdale Village",0.02,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.02
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.02,0.00


Create a new dataframe by adding the row values of "Japanese Restaurant" and "Sushi Restaurant" together. 

In [17]:
jpn_df = jpn.copy()
jpn_df['Japanese Restaurants'] = jpn['Japanese Restaurant'] + jpn['Sushi Restaurant']

jpn_df = jpn_df[['Neighborhood', 'Japanese Restaurants']]

jpn_df.head()

,Neighborhood,Japanese Restaurants
0,"Adelaide,King,Richmond",0.04
1,Berczy Park,0.03
2,"Brockton,Exhibition Place,Parkdale Village",0.02
3,Business Reply Mail Processing Centre 969 Eastern,0.02
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.02


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in terms of the frequency of Japanese Restaurants into 3 clusters.

In [18]:
# set number of clusters
kclusters = 3

clustering = jpn_df.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40]

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 0, 2, 2, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1], dtype=int32)

Append the cluster number to each neighborhood in the dataframe.

In [19]:
jpn_merged = jpn_df.copy()

# add clustering labels
jpn_merged["Cluster Labels"] = kmeans.labels_

jpn_merged.head()

,Neighborhood,Japanese Restaurants,Cluster Labels
0,"Adelaide,King,Richmond",0.04,0
1,Berczy Park,0.03,0
2,"Brockton,Exhibition Place,Parkdale Village",0.02,1
3,Business Reply Mail Processing Centre 969 Eastern,0.02,1
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.02,1


Merge the above dataframe with our previous dataframe 'toronto' that contains the geo information of all the neighborhoods.

In [20]:
jpn_merged = jpn_merged.join(toronto.set_index('Neighborhood'), on='Neighborhood')
jpn_merged.sort_values(['Cluster Labels'], inplace = True)

jpn_merged.head()

,Neighborhood,Japanese Restaurants,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",0.04,0,M5H,Downtown Toronto,43.650571,-79.384568
19,"Harbourfront East,Toronto Islands,Union Station",0.04,0,M5J,Downtown Toronto,43.640816,-79.381752
24,"Moore Park,Summerhill East",0.03,0,M4T,Central Toronto,43.689574,-79.383160
17,"Forest Hill North,Forest Hill West",0.05,0,M5P,Central Toronto,43.696948,-79.411307
16,"First Canadian Place,Underground city",0.03,0,M5X,Downtown Toronto,43.648429,-79.382280


Visualizing the resulting clusters.

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jpn_merged['Latitude'], jpn_merged['Longitude'], jpn_merged['Neighborhood'], jpn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters Select the Best Location

Examine each cluster to determine suitable locations for the restaurant.

First Cluster: Cluster 0

In [22]:
jpn_merged.loc[jpn_merged['Cluster Labels'] == 0]

,Neighborhood,Japanese Restaurants,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",0.04,0,M5H,Downtown Toronto,43.650571,-79.384568
19,"Harbourfront East,Toronto Islands,Union Station",0.04,0,M5J,Downtown Toronto,43.640816,-79.381752
24,"Moore Park,Summerhill East",0.03,0,M4T,Central Toronto,43.689574,-79.383160
17,"Forest Hill North,Forest Hill West",0.05,0,M5P,Central Toronto,43.696948,-79.411307
16,"First Canadian Place,Underground city",0.03,0,M5X,Downtown Toronto,43.648429,-79.382280
26,"Parkdale,Roncesvalles",0.03,0,M6R,West Toronto,43.648960,-79.456325
14,"Design Exchange,Toronto Dominion Centre",0.04,0,M5K,Downtown Toronto,43.647177,-79.381576
27,Rosedale,0.04,0,M4W,Downtown Toronto,43.679563,-79.377529
28,Roselawn,0.05,0,M5N,Central Toronto,43.711695,-79.416936
29,"Runnymede,Swansea",0.05,0,M6S,West Toronto,43.651571,-79.484450


Second Cluster: Cluster 1

In [23]:
jpn_merged.loc[jpn_merged['Cluster Labels'] == 1]

,Neighborhood,Japanese Restaurants,Cluster Labels,PostalCode,Borough,Latitude,Longitude
33,Studio District,0.01,1,M4M,East Toronto,43.659526,-79.340923
35,The Beaches,0.02,1,M4E,East Toronto,43.676357,-79.293031
23,"Little Portugal,Trinity",0.01,1,M6J,West Toronto,43.647927,-79.419750
18,"Harbord,University of Toronto",0.02,1,M5S,Downtown Toronto,43.662696,-79.400049
21,"High Park,The Junction South",0.02,1,M6P,West Toronto,43.661608,-79.464763
36,"The Beaches West,India Bazaar",0.01,1,M4L,East Toronto,43.668999,-79.315572
15,"Dovercourt Village,Dufferin",0.01,1,M6H,West Toronto,43.669005,-79.442259
13,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.02,1,M4V,Central Toronto,43.686412,-79.400049
8,Christie,0.02,1,M6G,Downtown Toronto,43.669542,-79.422564
7,"Chinatown,Grange Park,Kensington Market",0.02,1,M5T,Downtown Toronto,43.653206,-79.400049


Third Cluster: Cluster 2

In [24]:
jpn_merged.loc[jpn_merged['Cluster Labels'] == 2]

,Neighborhood,Japanese Restaurants,Cluster Labels,PostalCode,Borough,Latitude,Longitude
25,North Toronto West,0.08,2,M4R,Central Toronto,43.715383,-79.405678
12,Davisville North,0.07,2,M4P,Central Toronto,43.712751,-79.390197
11,Davisville,0.06,2,M4S,Central Toronto,43.704324,-79.388790
9,Church and Wellesley,0.09,2,M4Y,Downtown Toronto,43.665860,-79.383160
22,Lawrence Park,0.07,2,M4N,Central Toronto,43.728020,-79.388790


Observations: 

Four out of five Cluster 2 neighborhoods with the most concentration of Japanese restaurants are in Central Toronto area. Japanese restaurants in this area are very likely to suffer from intense competition. 

Most of the Cluster 0 neighborhoods with moderate saturation of Japanese restaurants are in Downtown Toronto. Downtown is always crowded with large population density with plenty of business opportunities. However, there are some immediate concerns when locating in a downtown area such as leasing cost and parking availability. 

Cluster 1 neighborhoods, mostly locate in East Toronto and West Toronto, have relatively low saturation of Japanese restaurants. Among these neighborhoods, two spots which are located in Downtown Toronto especially stand out. They are “Chinatown, Grange Park, Kensington Market” and “Harbord, University of Toronto”. The superior location with little concentration of Japanese restaurants may create advantageous opportunity for developing a new restaurant. 
